In [1]:
# 06/02/25 
# Gabriella Chan
# gabriella.chan@moansh.edu

# This script generates a memory mapped distance matrix using the brainsmash volume function
# It loads an atlas, takes coordinates of valid rois, and calls "volume" to generate the distance matrix

In [1]:
import nibabel as nib
import pandas as pd
import numpy as np
from brainsmash.workbench import volume
from brainsmash.mapgen.eval import sampled_fit
from brainsmash.mapgen.sampled import Sampled
import os

import time
from ttictoc import tic, toc

In [2]:
data_dir = './data/l_cx/'
out_dir = './results/l_cx/'

# Load atlas and brain map
atlas_img = nib.load('/fs03/kg98/gchan/Atlases/Tian/Schaefer_Tian/reordered/Schaefer2018_100Parcels_' +
    '7Networks_order_Tian_Subcortex_S2_MNI152NLin6Asym_1.5mm_reordered.nii.gz')
atlas = atlas_img.get_fdata()
img = nib.load('./data/lme_betas.nii.gz')
img_data = img.get_fdata()

roi_i = 1
roi_j = 50
mask = (atlas >= roi_i) & (atlas <= roi_j)

In [ ]:
gen_distmat = True
tic()
if gen_distmat:
    # Get the 3D coordinates and values of the masked image
    coords = np.column_stack(np.where(mask))
    coord_file = data_dir + 'coordinates.txt'
    np.savetxt(coord_file, coords, fmt='%d')

    gmv = img_data[np.where(mask)]
    brain_map = data_dir + 'lme_betas.txt'
    np.savetxt(brain_map, gmv, fmt='%f')

    # volume is a builtin BrainSMASH function that generates a memory-mapped distance matrix
    # This has a very long runtime, ~1 h for 200k voxels
    filenames = volume(coord_file, data_dir)
else:
    brain_map = data_dir + 'lme_betas.txt'
    filenames = {
        "D": data_dir + "distmat.npy",
        "index": data_dir + "index.npy"
    }
toc()

loading voxels coordinates from ./data/l_cx/coordinates.txt
file contains 156579 voxels
saving memory-mapped distance matrix files to ./data/l_cx/


2663.243753605522

In [4]:
# 13 min lh knn 1500, 14 min knn 2300, 13 tasks??
# 30 min l cx, p4 6 tasks

# These are three of the key parameters affecting the variogram fit
kwargs = {
    'ns': 500,
    'knn': 1500,
    'pv': 60,
    'kernel': 'gaussian',
    'n_jobs': 13
}

# Running this command will generate a matplotlib figure
# tic()
# sampled_fit(brain_map, filenames['D'], filenames['index'], nsurr=10, **kwargs)
# toc()

In [ ]:
import pickle
from sklearn.utils.validation import check_random_state

# 20 min 200k voxels, 13 tasks

try:
    with open(data_dir + 'gen.pkl', 'rb') as f:
        gen = pickle.load(f)
        # reset random state in generator
        gen._rs = check_random_state(None)
except FileNotFoundError:
    gen = Sampled(x=brain_map, D=filenames['D'], index=filenames['index'], **kwargs)
    with open(data_dir + 'gen.pkl', 'wb') as f:
        pickle.dump(gen, f)


In [14]:
import pickle
from sklearn.utils.validation import check_random_state
tic()

name = "l_cx"
# 20 min 200k voxels, 13 tasks

print("Searching for generator for " + name)
try:
    with open(data_dir + 'gen.pkl', 'rb') as f:
        gen = pickle.load(f)
        # reset random state in generator
        gen._rs = check_random_state(None)
        print("Generator found for " + name)
except FileNotFoundError:
    print("Generator not found. Creating generator for " + name)
    gen = Sampled(x=brain_map, D=filenames['D'], index=filenames['index'], **kwargs)
    with open(data_dir + 'gen.pkl', 'wb') as f:
        pickle.dump(gen, f)
    print("Generator created for " + name)


toc()

Searching for generator for l_cx
Generator found for l_cx


0.7075775200501084

In [15]:
# There is no economies of scale in generating multiple surrogate maps
# 4h for 1 null map of 166k voxels

tic()
n_maps = 1
surrogate_maps = gen(n=n_maps)
print(surrogate_maps.shape)
toc()

(156579,)


13.373763739131391

In [16]:
def save_map(null_map, out_dir, filename):
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    modified_img = nib.Nifti1Image(null_map, img.affine, img.header)
    nib.save(modified_img, out_dir + filename)

In [17]:
null_map = np.zeros_like(img_data, dtype=float)
if n_maps == 1:
    null_map[mask] = surrogate_maps
    save_map(null_map, out_dir, "test2.nii.gz")
else:
    for i in range(n_maps):
        null_map[mask] = surrogate_maps[i]
        save_map(null_map, out_dir, f"null_map_{i}.nii.gz")

# np.savetxt(out_dir + 'mask.csv', mask, delimiter=',', fmt='%d')
# np.savetxt(out_dir + 'surrogate_maps.csv', surrogate_maps, delimiter=',', fmt='%f')